In [ ]:
# based on code from https://medium.com/dataseries/convolutional-autoencoder-in-pytorch-onencoded_space_dim-mnist-dataset-d65145c132ac 

In [ ]:
import matplotlib.pyplot as plt # plotting library
import numpy as np # this module is useful to work with numerical arrays
import pandas as pd 
import random, os
import torch
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader,random_split
from torch import nn
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
# Check if the GPU is available
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

# dataset constants:
MAPS_AND_PATHS_DATASET = False              # set True if the dataset you are loading maps from contains both maps and paths (ex. random_30_density), set False if dataset only contains maps (ex. 200000_maps)
DATASET = '200000_maps'
CALC_COVERAGE = False                       # coverage of training dataset will be calculated if True

# encoder constants:
FEATURE_ARR = [16,32,64,128]

# training constants:
SAVE = True
ENCODER_PATH = "checkpoints/encoder/"
DECODER_PATH = "checkpoints/decoder/"
LR = 0.000025
BATCH_SIZE = 128
NUM_EPOCHS = 30

In [ ]:
if MAPS_AND_PATHS_DATASET:          # set data loader to load maps from a dataset that contains both maps and paths
    class MapsDataset(torch.utils.data.Dataset):
        # init the dataset, shape = L x W
        def __init__(self, gen_dir, transform=None, shape = (100,100), device='cpu'):
            self.device = device
            self.maps = []

            num_maps = 0
            for directory in os.listdir(gen_dir):
                num_maps += 1
                dir_path = os.path.join(gen_dir, directory)

                # open map file:
                with open(f"{dir_path}/{directory}.txt", 'r') as f:
                    self.flat_map = np.loadtxt(f)
                    self.map = np.asarray(self.flat_map, dtype=np.float32).reshape(shape[0],shape[1])

                    # add map to maps array:
                    self.maps.append(self.map)
            
            self.transform = transform
            print("Done!")

        def __len__(self):
            return len(self.maps)

        def __getitem__(self, idx):
            x = self.maps[idx]
            if self.transform:
                x = self.transform(x)
            
            return x

else:                                   # set data loader to load maps from a dataset with only maps
    class MapsDataset(torch.utils.data.Dataset):
        def __init__(self, map_dir, transform=None, shape = (100,100)):
            self.maps = []                                                  # create a list to hold all paths read from file
            for filename in os.listdir(map_dir):
                with open(os.path.join(map_dir, filename), 'r') as f:                               # open in readonly mode
                    self.flat_map = np.loadtxt(f)                                                   # load in the flat path from file
                    self.map = np.asarray(self.flat_map, dtype=int).reshape(shape[0], shape[1])     # unflatten the map from the file
                    self.maps.append(self.map)                                                      # add the path to paths list

            self.transform = transform
            print("Done!")

        def __len__(self):
            return len(self.maps)

        def __getitem__(self, idx):
            x = np.float32(self.maps[idx])

            return x

In [ ]:
data_transform = transforms.Compose([
transforms.ToTensor(),
])

if MAPS_AND_PATHS_DATASET:
    train_dataset = MapsDataset(gen_dir = f"./env/{DATASET}/train/", shape = (64,64), transform=data_transform)
    test_dataset = MapsDataset(gen_dir = f"./env/{DATASET}/eval/", shape = (64,64), transform=data_transform)
else:
    dataset = MapsDataset(map_dir = f"./env/{DATASET}/", shape = (64,64), transform=data_transform)
    m=len(dataset)
    train_dataset, test_dataset = random_split(dataset, [int(m-m*0.05), int(m*0.05)])

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE,shuffle=True)

In [ ]:
if CALC_COVERAGE:
    # determine train dataset coverage %:
    perc_sum = 0
    for image_batch in train_loader:
        
        obs_coords = np.argwhere(torch.tensor(image_batch)==1)
        obs_perc = (len(obs_coords)/(64*64))
        perc_sum += obs_perc

        # # if you want to see which points count as 'coverage':

        # obs_coords = np.transpose(torch.flipud(obs_coords))
        # plt.imshow(image_batch)
        # plt.scatter(obs_coords[:,0], obs_coords[:,1])
        # plt.show()
        # break

    average_coverage = perc_sum/len(train_loader)*100
    print(f"Average coverage: {round(average_coverage, 2)}%")

In [ ]:
class Encoder(nn.Module):
    
    def __init__(self):
        super().__init__()
        
        ### Convolutional section
        self.encoder_cnn = nn.Sequential(
            nn.Conv2d(1, FEATURE_ARR[0], 5, stride=2, padding=1),
            nn.BatchNorm2d(FEATURE_ARR[0]),
            nn.ReLU(True),
            nn.Conv2d(FEATURE_ARR[0], FEATURE_ARR[1], 5, stride=2, padding=1),
            nn.BatchNorm2d(FEATURE_ARR[1]),
            nn.ReLU(True),
            nn.Conv2d(FEATURE_ARR[1], FEATURE_ARR[2], 5, stride=2, padding=1),
            nn.BatchNorm2d(FEATURE_ARR[2]),
            nn.ReLU(True),     
            nn.Conv2d(FEATURE_ARR[2], FEATURE_ARR[3], 7, stride=2, padding=0),
            nn.BatchNorm2d(FEATURE_ARR[3]),
            nn.ReLU(True),  
        )
        
        ### Flatten layer
        self.flatten = nn.Flatten(start_dim=1)
        
    def forward(self, x):
        x = self.encoder_cnn(x)
        x = self.flatten(x)
        return x

In [ ]:
class Decoder(nn.Module):
    
    def __init__(self):
        super().__init__()

        self.unflatten = nn.Unflatten(dim=1, 
        unflattened_size=(FEATURE_ARR[3], 1, 1))

        self.decoder_conv = nn.Sequential(
            nn.ConvTranspose2d(FEATURE_ARR[3], FEATURE_ARR[2], 7, 
            stride=2, padding=0, output_padding=0),
            nn.BatchNorm2d(FEATURE_ARR[2]),
            nn.ReLU(True),
            nn.ConvTranspose2d(FEATURE_ARR[2], FEATURE_ARR[1], 5, 
            stride=2, padding=1, output_padding=0),
            nn.BatchNorm2d(FEATURE_ARR[1]),
            nn.ReLU(True),
            nn.ConvTranspose2d(FEATURE_ARR[1], FEATURE_ARR[0], 5, 
            stride=2, padding=1, output_padding=0),
            nn.BatchNorm2d(FEATURE_ARR[0]),
            nn.ReLU(True),
            nn.ConvTranspose2d(FEATURE_ARR[0], 1, 5, 
            stride=2, padding=1, output_padding=1)
        )
        
    def forward(self, x):
        x = self.unflatten(x)
        x = self.decoder_conv(x)
        x = torch.sigmoid(x)
        return x

In [ ]:
### Define the loss function
loss_fn = torch.nn.MSELoss()

### Set the random seed for reproducible results
torch.manual_seed(0)

encoder = Encoder()
decoder = Decoder()
params_to_optimize = [
    {'params': encoder.parameters()},
    {'params': decoder.parameters()}
]

optim = torch.optim.Adam(params_to_optimize, lr=LR)

# Move both the encoder and the decoder to the selected device
encoder.to(device)
decoder.to(device)

In [ ]:
### Training function
def train_epoch(encoder, decoder, device, dataloader, loss_fn, optimizer):
    # Set train mode for both the encoder and the decoder
    encoder.train()
    decoder.train()
    train_loss = []
    # Iterate the dataloader
    for batch_index, image_batch in enumerate(dataloader):
        
        # ensure proper dimensions:
        if not MAPS_AND_PATHS_DATASET:
            image_batch = image_batch[:,None,:,:]

        # Move tensor to the proper device
        image_batch = image_batch.to(device)
        # Encode data
        encoded_data = encoder(image_batch)
        # Decode data
        decoded_data = decoder(encoded_data)
        # Evaluate loss
        loss = loss_fn(decoded_data, image_batch)
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss.append(loss.detach().cpu().numpy())

    return np.mean(train_loss)

In [ ]:
### Testing function
def test_epoch(encoder, decoder, device, dataloader, loss_fn):
    # Set evaluation mode for encoder and decoder
    encoder.eval()
    decoder.eval()
    with torch.no_grad(): # No need to track the gradients
        # Define the lists to store the outputs for each batch
        conc_out = []
        conc_label = []
        for batch_index, image_batch in enumerate(dataloader):
            
            # ensure proper dimensions:
            if not MAPS_AND_PATHS_DATASET:
                image_batch = image_batch[:,None,:,:]

            # Move tensor to the proper device
            image_batch = image_batch.to(device)
            # Encode data
            encoded_data = encoder(image_batch)
            # Decode data
            decoded_data = decoder(encoded_data)
            # Append the network output and the original image to the lists
            conc_out.append(decoded_data.cpu())
            conc_label.append(image_batch.cpu())
        # Create a single tensor with all the values in the lists
        conc_out = torch.cat(conc_out)
        conc_label = torch.cat(conc_label) 
        # Evaluate global loss
        val_loss = loss_fn(conc_out, conc_label)
    return val_loss.data

In [ ]:
def plot_ae_outputs(encoder,decoder,n=10):
    plt.figure(figsize=(16,4.5))
    for i in range(n):
      ax = plt.subplot(2,n,i+1)
      
      # ensure proper dimensions:
      if MAPS_AND_PATHS_DATASET:
        img = torch.tensor(test_dataset[i]).unsqueeze(0).to(device)
      else:
        img = torch.tensor(test_dataset[i]).unsqueeze(0).unsqueeze(0).to(device)

      encoder.eval()
      decoder.eval()
      with torch.no_grad():
         rec_img  = decoder(encoder(img))
      plt.imshow(img.cpu().squeeze().numpy(), cmap='gist_gray')
      ax.get_xaxis().set_visible(False)
      ax.get_yaxis().set_visible(False)  
      if i == n//2:
        ax.set_title('Original images')
      ax = plt.subplot(2, n, i + 1 + n)
      plt.imshow(rec_img.cpu().squeeze().numpy(), cmap='gist_gray')  
      ax.get_xaxis().set_visible(False)
      ax.get_yaxis().set_visible(False)  
      if i == n//2:
         ax.set_title('Reconstructed images')
    plt.show()   

In [ ]:
diz_loss = {'train_loss':[],'val_loss':[]}
for epoch in range(NUM_EPOCHS):
   train_loss = train_epoch(encoder,decoder,device,train_loader,loss_fn,optim)
   val_loss = test_epoch(encoder,decoder,device,test_loader,loss_fn)
   print('\n EPOCH {}/{} \t train loss {} \t val loss {}'.format(epoch + 1, NUM_EPOCHS,train_loss,val_loss))
   diz_loss['train_loss'].append(train_loss)
   diz_loss['val_loss'].append(val_loss)
   if epoch % 1 == 0:
      plot_ae_outputs(encoder,decoder,n=10)
plot_ae_outputs(encoder,decoder,n=10)

In [ ]:
# Plot losses
plt.figure(figsize=(10,8))
plt.semilogy(diz_loss['train_loss'], label='Train')
plt.semilogy(diz_loss['val_loss'], label='Valid')
plt.xlabel('Epoch')
plt.ylabel('Average Loss')
plt.legend()
plt.title('Run Loss')
plt.show()

In [ ]:
# save encoder and decoder

if SAVE:
    torch.save({
                'model_state_dict': encoder.state_dict(),
    }, f"{ENCODER_PATH}epoch-{epoch}.tar")
    torch.save({
                'model_state_dict': decoder.state_dict(),
    }, f"{DECODER_PATH}epoch-{epoch}.tar")

In [ ]:
# example using encoder/decoder on loaded map

flat_map = np.loadtxt("env/test_map.txt", skiprows=2)                                                   # load in the flat path from file
map = torch.tensor(np.asarray(flat_map, dtype=int).reshape(64, 64)).to(device, dtype=torch.float)     # unflatten the map from the file

plt.imshow(map.cpu())
plt.show()

map_4d = map.unsqueeze(0).unsqueeze(0)

encoded_map = encoder(map_4d)
decoded_map = decoder(encoded_map)

new_map_2d = decoded_map.squeeze()

plt.imshow(new_map_2d.cpu().detach().numpy())
plt.show()